# **Preprocess the Video**

In [ ]:
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt

import skimage
from PIL import Image
from skimage.io import imread, imshow, imread_collection, concatenate_images
from skimage.transform import resize
from skimage.util import crop, pad
from skimage.morphology import label
from skimage.color import rgb2gray, gray2rgb

import os

import zipfile
z = zipfile.ZipFile("Dancer_Images.zip", "w")

In [ ]:
cap = cv2.VideoCapture('/content/Dance_Video.avi')
print(cap.get(cv2.CAP_PROP_FPS))

30.0


In [ ]:
%%time


try:
  if not os.path.exists('data'):
      os.makedirs('data')
except OSError:
  print ('Error: Creating directory of data')

currentFrame = 0
count = 0
TRAIN_SIZE = 27000
FRAME_SKIP = 2
IMG_WIDTH = 96
IMG_HEIGHT = 64
IMG_CHANNELS = 1
X_train = np.zeros((TRAIN_SIZE, IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype='float32')

video = cv2.VideoWriter('Simple_Shadow_Dancer_Video.avi',cv2.VideoWriter_fourcc(*"MJPG"), 30, (IMG_WIDTH, IMG_HEIGHT), False)

while(count < TRAIN_SIZE):
  try:
      ret, frame = cap.read()

      if currentFrame % FRAME_SKIP == 0:
          count += 1
          if count % int(TRAIN_SIZE/10) == 0:
              print(str((count/TRAIN_SIZE)*100)+"% done")
          # preprocess frames
          img = frame
          img = rgb2gray(img)
          img = resize(img, (IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), mode='constant', preserve_range=True)
          img[img > 0.2] = 255
          img[img <= 0.2] = 0
          # save frame to zip and new video sample
          name = './data/frame' + str(count) + '.jpg'
          cv2.imwrite(name, img)
          video.write(gray2rgb(img.astype('uint8')))
          z.write(name)
          os.remove(name)
          # save image to training set if training directly to part 2
          img = img.astype('float32') / 255.
          X_train[count] = img
  except:
      print('Frame error')
      break
  currentFrame += 1

print(str(count)+" Frames collected")
cap.release()
z.close()
video.release()

Frame error
1464 Frames collected
CPU times: user 2min 23s, sys: 1min 44s, total: 4min 8s
Wall time: 2min 10s
